In [ ]:
#This python example shows how to retrieve data for your pedestrian and cycle counter(s) using the REST API of InfoQus
#The original example written for and tested in Python v.3.6.5
#The example uses the python 'requests' module, for more info on this module visit http://docs.python-requests.org/en/master/

# blurb from Infoqus manual at - https://q-free.atlassian.net/servicedesk/customer/portal/2/topic/a0e5a6d3-b6ee-40e7-8857-9797642e1090/article/1142063109
# original with mninor adaptation to suit own needs

# (open) data available at: https://usmart.io/org/cyclingscotland 
# to download detail via API request client_id 

## application STA Climate Change 
## Aba-Sah Dadzie, last edited 27.08.2021


In [1]:
import requests
import shutil
import sys
import json

from datetime import datetime, timedelta, date
from requests import Request, Session


In [2]:
def get_api_bearer_key(apiclient_id, apiclient_secret):
    
    getbearer_keyurl = 'https://api-identity.q-freehub.com/oauth/token'
    
    getbearer_keybody = {'grant_type': 'client_credentials', 
                        'client_id': apiclient_id, 
                        'client_secret': apiclient_secret, 
                        'audience': 'urn:q-freehub-api'
                       }   
    
    bearerTokenRequet = requests.post(getbearer_keyurl, json = getbearer_keybody)
    
    if bearerTokenRequet.status_code != 200:
        print(bearerTokenRequet.url)   
        print("Error " + str(bearerTokenRequet.status_code))
        return None
    else:
        rjson = bearerTokenRequet.json()
        return rjson['access_token']
    

In [15]:
 
client_id = 'requested_client_id'
client_secret = 'requested_client_secret'

#base address of InfoQus api
infoqus_api_base_url = 'https://api-infoqus.q-freehub.com/api/v1/''

output_path = 'infoqus_output/'

In [ ]:
# valid for a 24-hour period

bearer_key = get_api_bearer_key(client_id, client_secret)
bearer_key


## Function definitions

In [5]:
 
def call_api(apiurl, bearer_key, parameters = None):
    if parameters == None:
        request = requests.get(apiurl, 
                         headers = {'accept': 'application/json', 'Authorization': 'Bearer ' + bearer_key})
    else:
        request = requests.get(apiurl, 
                         headers = {'accept': 'application/json', 'Authorization': 'Bearer ' + bearer_key}, 
                         params = parameters)
        
    if request.status_code != 200:
        print(request.url)   
        print("Error " + str(request.status_code))
        return None
    else:
        rjson = request.json()
        return rjson
 

In [6]:
def getSiteData(apibaseurl, bearerKey, siteId = None):
    return call_api(apibaseurl + 'sites' + (('/' + str(siteId)) if (siteId != None) else ''), bearerKey)   
 

In [7]:
# get all vehicles (pedestrian + bicycle + unknown) from API endpoint <apiurl>/Reporting/SingleSite?fromDate=<date1>&toDate=<date2>

def getAllCountsByDay(apiBaseUrl, bearerKey, siteId, fromDate, toDate):

    if (isinstance(fromDate, str)):
        fromDate = datetime.strptime(fromDate, '%d/%m/%Y')
    if (isinstance(toDate, str)):
        toDate = datetime.strptime(toDate, '%d/%m/%Y')

    params = {'fromDate': fromDate, 
              'toDate': toDate
             }
    
    return call_api(apiBaseUrl + "Reporting/SingleSite/" + str(siteId), bearerKey, params)
   

In [8]:
 
# get data in defined intervals - API endpoint <apiurl>/Reporting/CrossingCount
# supports params fromDate, toDate, siteIds, interval, classes, lanes, directions, aggregations (not interested but returns regardless)

def getCounterData(apiBaseUrl, bearerKey, 
                   fromDate, toDate, 
                   countInterval = 'hour', 
                   vehicleClasses = 'Bicycle', 
                   siteIds = None, 
                   trafficDirections = None, 
                   laneIds = None):

    if (isinstance(fromDate, str)):
        fromDate = datetime.strptime(fromDate, '%d/%m/%Y')
    if (isinstance(toDate, str)):
        toDate = datetime.strptime(toDate, '%d/%m/%Y')


    params = {'fromDate': fromDate,
              'toDate': toDate,
              'siteIds': siteIds,
              'interval': countInterval, # ('hour', 'day', 'week', 'month' and 'year)
              'classes': vehicleClasses, # ('Unknown', 'Pedestrian, Bicycle')
              'directions': trafficDirections, # ('NorthBound', 'SouthBound', 'EastBound', 'WestBound')
              'lanes': laneIds # (0, 1, 2, 3)
             }
    
    return call_api(apiBaseUrl + "Reporting/CrossingCount", bearerKey, params)
 

In [9]:
def getDetailedSiteCounterData(apiBaseUrl, bearerKey, outputPath, fromDate, toDate, siteId, countInterval = 'hour', 
                               vehicleClasses = 'Bicycle', trafficDirections = None, laneIds = None):

    data_as_json = getCounterData(apiBaseUrl, bearerKey, 
                                  fromDate, toDate, countInterval,
                                  vehicleClasses, siteId, trafficDirections, laneIds)      

    if (data_as_json != None):
        extractCountDataToFile(data_as_json, outputPath, fromDate, toDate, siteId, countInterval, 
                               vehicleClasses, trafficDirections, laneIds)                
    #else:
        # error will print ... 


In [10]:
 
def getDetailedCounterData(apiBaseUrl, bearerKey, outputPath, fromDate, toDate, countInterval = 'hour', 
                           vehicleClasses = 'Bicycle', trafficDirections = None, laneIds = None):

    reporting_sites = getSiteData(apiBaseUrl, bearerKey)

    if reporting_sites != None:

        for site in reporting_sites:
            print('\nSite ID: ' + str(site['id']))

            data_as_json = getCounterData(apiBaseUrl, bearerKey, 
                                          fromDate, toDate, countInterval,
                                          vehicleClasses, site['id'], trafficDirections, laneIds)   
            
            if (data_as_json != None):                
                extractCountDataToFile(data_as_json, outputPath, fromDate, toDate, site['id'], countInterval, 
                                       vehicleClasses, trafficDirections, laneIds)                
            #else:
                # error will print ... 
                
        # end for
    # end if
# end function

In [11]:
def extractCountDataToFile(data_as_json, outputPath, fromDate, toDate, siteId, countInterval = 'hour', 
                           vehicleClasses = 'Bicycle', trafficDirections = None, laneIds = None):
    
    if (isinstance(fromDate, str)):
        fromDate = datetime.strptime(fromDate, '%d/%m/%Y')
    if (isinstance(toDate, str)):
        toDate = datetime.strptime(toDate, '%d/%m/%Y')

    
    fileName = (outputPath +  'site' + str(siteId) + '-' + countInterval + '-' + 
                    #str(int(fromDate.timestamp())) + '-' + str(int(toDate.timestamp())) + '-' + 
                    str(int(fromDate.strftime("%d%m%Y"))) + '-' + str(int(toDate.strftime("%d%m%Y"))) + '-' +                     
                    vehicleClasses + '-' + 
                    ('' if (trafficDirections == None) else (trafficDirections + '-')) + 
                    ('' if(laneIds == None) else laneIds)  + 
                    '.json')
    print('Writing data to file: ' + fileName + ' ...')

    dataFile = open(fileName, 'w')
    dataFile.write(json.dumps(data_as_json))
    dataFile.close()


In [12]:
def getHourlyCounterDataAcrossYears(apiBaseUrl, bearerKey, outputPath, fromDate, toDate, vehicleClasses = 'Bicycle', 
                                    siteId = None, trafficDirections = None, laneIds = None):

    fromDate = datetime.strptime(fromDate, '%d/%m/%Y')
    toDate = datetime.strptime(toDate, '%d/%m/%Y')

    startYear = fromDate.year
    endYear = toDate.year
    currentYear = startYear 
    print(startYear, endYear, currentYear, (currentYear <= endYear))
    
    while (currentYear <= endYear):

        print(currentYear)

        if ((currentYear == startYear) & (currentYear == endYear)):
            spanStart = fromDate
            spanEnd = toDate
            print("span:", spanStart, spanEnd, "\n")
            
        
        elif (currentYear == startYear):

            spanStart = fromDate
            spanEnd = fromDate.replace(year = fromDate.year, month = 12, day = 31)
            print("start period:", spanStart, spanEnd, "\n")


        elif (currentYear == endYear) :

            spanStart = spanStart.replace(year = spanStart.year + 1, month = 1, day = 1)
            spanEnd = toDate
            print("final period:", spanStart, spanEnd, "\n")


        else:

            spanStart = spanStart.replace(year = spanStart.year + 1, month = 1, day = 1)
            spanEnd = spanEnd.replace(year = spanEnd.year + 1)
            print("interim period:", spanStart, spanEnd, "\n")


        # extract counts
        if (siteId == None):
            getDetailedCounterData(infoqus_api_base_url, bearer_key, outputPath, spanStart, spanEnd, 
                                   vehicleClasses = vehicleClasses, trafficDirections = trafficDirections, laneIds = laneIds)
        else:
            getDetailedSiteCounterData(infoqus_api_base_url, bearer_key, outputPath, spanStart, spanEnd, siteId, 
                                       vehicleClasses = vehicleClasses, trafficDirections = trafficDirections, laneIds = laneIds)            

        # end loop
        currentYear = currentYear + 1 
    
    

## Examples - request data at different levels of detail

In [9]:
getSiteData(infoqus_api_base_url, bearer_key, 12)

{'id': 12,
 'status': 'Offline',
 'batteryLevel': 6743.0,
 'lastConnected': '2021-04-17T00:10:43.6984008+00:00',
 'type': 'Cmu',
 'externalId': '6047',
 'name': 'EDU - Bearsden, Craigdhu Burn',
 'geometry': {'type': 'Point', 'coordinates': [-4.317793, 55.931855]},
 'offlineAfterMinutes': 3600,
 'siteClassMappings': [{'fromClassIndex': 70, 'toVehicleClass': 'Bicycle'},
  {'fromClassIndex': 71, 'toVehicleClass': 'Bicycle'},
  {'fromClassIndex': 123, 'toVehicleClass': 'Pedestrian'},
  {'fromClassIndex': 72, 'toVehicleClass': 'Bicycle'}]}

In [17]:
reporting_sites = getSiteData(infoqus_api_base_url, bearer_key)


dataFile = open(output_path + "reporting_sites.json", 'w')
dataFile.write(json.dumps(reporting_sites))
dataFile.close()


reporting_sites[:3]

[{'id': 5,
  'status': 'Offline',
  'batteryLevel': 6582.0,
  'lastConnected': '2021-08-27T02:01:07.7220602+00:00',
  'type': 'Cmu',
  'externalId': '4918',
  'name': 'FIF - Dunfermline, Kingseat Road',
  'geometry': {'type': 'Point', 'coordinates': [-3.4282, 56.08681]},
  'offlineAfterMinutes': 40,
  'siteClassMappings': [{'fromClassIndex': 70, 'toVehicleClass': 'Bicycle'},
   {'fromClassIndex': 123, 'toVehicleClass': 'Pedestrian'},
   {'fromClassIndex': 71, 'toVehicleClass': 'Bicycle'},
   {'fromClassIndex': 72, 'toVehicleClass': 'Bicycle'}]},
 {'id': 7,
  'status': 'Pending',
  'type': 'Cmu',
  'externalId': '123456789',
  'name': 'Shields Road at Scotland Street',
  'geometry': {'type': 'Point', 'coordinates': [-4.2764, 55.8498]},
  'offlineAfterMinutes': 3600,
  'siteClassMappings': [{'fromClassIndex': 70, 'toVehicleClass': 'Bicycle'},
   {'fromClassIndex': 123, 'toVehicleClass': 'Pedestrian'}]},
 {'id': 8,
  'status': 'Online',
  'batteryLevel': 6534.0,
  'lastConnected': '2021-0

In [12]:
getAllCountsByDay(infoqus_api_base_url, bearer_key, 12, '18/02/2021', '20/02/2021')


# item['class'] == 0: Unknown 
# item['class'] == 1: Pedestrian
# item['class'] == 2: Bicycle

[{'date': '2021-02-18T00:00:00',
  'data': [{'class': 0, 'count': 9},
   {'class': 1, 'count': 179},
   {'class': 2, 'count': 7}]},
 {'date': '2021-02-19T00:00:00',
  'data': [{'class': 0, 'count': 10},
   {'class': 1, 'count': 193},
   {'class': 2, 'count': 19}]},
 {'date': '2021-02-20T00:00:00',
  'data': [{'class': 0, 'count': 20},
   {'class': 1, 'count': 321},
   {'class': 2, 'count': 15}]}]

In [14]:
getCounterData(infoqus_api_base_url, bearer_key, '18/02/2021', '19/02/2021',
               countInterval = 'hour', 
               vehicleClasses = ('Pedestrian, Bicycle'), 
               #siteIds = 12, 
               trafficDirections = ('NorthBound', 'SouthBound', 'EastBound', 'WestBound'),
               laneIds = (0, 1, 2, 3)
              )
          

{'crossingCountPerTimeInterval': [{'localTimestamp': '2021-02-18T00:00:00',
   'count': 222},
  {'localTimestamp': '2021-02-18T01:00:00', 'count': 270},
  {'localTimestamp': '2021-02-18T02:00:00', 'count': 759},
  {'localTimestamp': '2021-02-18T03:00:00', 'count': 632},
  {'localTimestamp': '2021-02-18T04:00:00', 'count': 519},
  {'localTimestamp': '2021-02-18T05:00:00', 'count': 683},
  {'localTimestamp': '2021-02-18T06:00:00', 'count': 860},
  {'localTimestamp': '2021-02-18T07:00:00', 'count': 1393},
  {'localTimestamp': '2021-02-18T08:00:00', 'count': 1498},
  {'localTimestamp': '2021-02-18T09:00:00', 'count': 1682},
  {'localTimestamp': '2021-02-18T10:00:00', 'count': 2374},
  {'localTimestamp': '2021-02-18T11:00:00', 'count': 1388},
  {'localTimestamp': '2021-02-18T12:00:00', 'count': 771},
  {'localTimestamp': '2021-02-18T13:00:00', 'count': 1214},
  {'localTimestamp': '2021-02-18T14:00:00', 'count': 1673},
  {'localTimestamp': '2021-02-18T15:00:00', 'count': 1710},
  {'localTime

In [15]:
getCounterData(infoqus_api_base_url, bearer_key, '18/02/2021', '29/03/2021',
               countInterval = 'week', 
               #vehicleClasses = ('Pedestrian, Bicycle'), 
               siteIds = 12, 
               trafficDirections = ('NorthBound', 'SouthBound', 'EastBound', 'WestBound'),
               laneIds = (0, 1, 2, 3)
              )
          

{'crossingCountPerTimeInterval': [{'localTimestamp': '2021-02-15T00:00:00',
   'count': 76},
  {'localTimestamp': '2021-02-22T00:00:00', 'count': 173},
  {'localTimestamp': '2021-03-01T00:00:00', 'count': 193},
  {'localTimestamp': '2021-03-08T00:00:00', 'count': 99},
  {'localTimestamp': '2021-03-15T00:00:00', 'count': 199},
  {'localTimestamp': '2021-03-22T00:00:00', 'count': 136},
  {'localTimestamp': '2021-03-29T00:00:00', 'count': 20}],
 'aggregations': {'total': 896}}

In [16]:
getCounterData(infoqus_api_base_url, bearer_key, '18/02/2021', '29/03/2021',
               countInterval = 'week', 
               vehicleClasses = 'Pedestrian', 
               siteIds = 12, 
               trafficDirections = ('NorthBound', 'SouthBound', 'EastBound', 'WestBound'),
               laneIds = (0, 1, 2, 3)
              )
          

{'crossingCountPerTimeInterval': [{'localTimestamp': '2021-02-15T00:00:00',
   'count': 1264},
  {'localTimestamp': '2021-02-22T00:00:00', 'count': 2482},
  {'localTimestamp': '2021-03-01T00:00:00', 'count': 2658},
  {'localTimestamp': '2021-03-08T00:00:00', 'count': 1862},
  {'localTimestamp': '2021-03-15T00:00:00', 'count': 2705},
  {'localTimestamp': '2021-03-22T00:00:00', 'count': 2160},
  {'localTimestamp': '2021-03-29T00:00:00', 'count': 264}],
 'aggregations': {'total': 13395}}

In [26]:
getDetailedSiteCounterData(infoqus_api_base_url, bearer_key, output_path, '01/01/2021', '14/05/2021',
                       countInterval = 'week', 
                       # vehicleClasses = 'Pedestrian', 
                       siteId = 12, 
                       trafficDirections = 'SouthBound', #'EastBound', 'WestBound'),
                       #laneIds = (0, 1, 2, 3)
                          )


Writing data to file: infoqus_output_plus/site12-week-1012021-14052021-Bicycle-SouthBound-.json ...


In [25]:
          
getCounterData(infoqus_api_base_url, bearer_key, '18/01/2017', '19/01/2017',
               countInterval = 'hour', 
               vehicleClasses = ('Pedestrian, Bicycle'), 
               #siteIds = 12, 
               trafficDirections = ('NorthBound', 'SouthBound', 'EastBound', 'WestBound'),
               laneIds = (0, 1, 2, 3)
              )
              

{'crossingCountPerTimeInterval': [{'localTimestamp': '2017-01-18T00:00:00',
   'count': 0},
  {'localTimestamp': '2017-01-18T01:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T02:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T03:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T04:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T05:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T06:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T07:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T08:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T09:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T10:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T11:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T12:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T13:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T14:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T15:00:00', 'count': 0},
  {'localTimestamp': '2017-01-18T16:00:00', 'count': 

In [28]:
          
getCounterData(infoqus_api_base_url, bearer_key, '01/06/2021', '30/06/2021',
               countInterval = 'day', 
               vehicleClasses = ('Pedestrian, Bicycle'), 
               #siteIds = 12, 
               trafficDirections = ('NorthBound', 'SouthBound', 'EastBound', 'WestBound'),
               laneIds = (0, 1, 2, 3)
              )
              

{'crossingCountPerTimeInterval': [{'localTimestamp': '2021-06-01T00:00:00',
   'count': 40102},
  {'localTimestamp': '2021-06-02T00:00:00', 'count': 41190},
  {'localTimestamp': '2021-06-03T00:00:00', 'count': 50263},
  {'localTimestamp': '2021-06-04T00:00:00', 'count': 43526},
  {'localTimestamp': '2021-06-05T00:00:00', 'count': 37733},
  {'localTimestamp': '2021-06-06T00:00:00', 'count': 41735},
  {'localTimestamp': '2021-06-07T00:00:00', 'count': 38965},
  {'localTimestamp': '2021-06-08T00:00:00', 'count': 34966},
  {'localTimestamp': '2021-06-09T00:00:00', 'count': 27304},
  {'localTimestamp': '2021-06-10T00:00:00', 'count': 39131},
  {'localTimestamp': '2021-06-11T00:00:00', 'count': 37532},
  {'localTimestamp': '2021-06-12T00:00:00', 'count': 53329},
  {'localTimestamp': '2021-06-13T00:00:00', 'count': 43039},
  {'localTimestamp': '2021-06-14T00:00:00', 'count': 44117},
  {'localTimestamp': '2021-06-15T00:00:00', 'count': 34114},
  {'localTimestamp': '2021-06-16T00:00:00', 'count

## Batch downloads

In [ ]:
#output_path = 'infoqus_output_plus/'
#
#getHourlyCounterDataAcrossYears(infoqus_api_base_url, bearer_key, output_path, '01/06/2017', '31/05/2021', 
#                                trafficDirections = 'NorthBound') 
#
#getHourlyCounterDataAcrossYears(infoqus_api_base_url, bearer_key, output_path, '01/06/2017', '31/05/2021', 
#                                trafficDirections = 'SouthBound') 
#
#getHourlyCounterDataAcrossYears(infoqus_api_base_url, bearer_key, output_path, '01/06/2017', '31/05/2021', 
#                                trafficDirections = 'WestBound')
#
#getHourlyCounterDataAcrossYears(infoqus_api_base_url, bearer_key, output_path, '01/06/2017', '31/05/2021', 
#                                trafficDirections = 'EastBound')
#    
#    
#    
#getHourlyCounterDataAcrossYears(infoqus_api_base_url, bearer_key, output_path, '01/06/2017', '31/05/2021', 
#                                vehicleClasses = 'Pedestrian', trafficDirections = 'NorthBound') 
#
#getHourlyCounterDataAcrossYears(infoqus_api_base_url, bearer_key, output_path, '01/06/2017', '31/05/2021', 
#                                vehicleClasses = 'Pedestrian', trafficDirections = 'SouthBound') 
#
#getHourlyCounterDataAcrossYears(infoqus_api_base_url, bearer_key, output_path, '01/06/2017', '31/05/2021', 
#                                vehicleClasses = 'Pedestrian', trafficDirections = 'WestBound')
#
#getHourlyCounterDataAcrossYears(infoqus_api_base_url, bearer_key, output_path, '01/06/2017', '31/05/2021', 
#                                vehicleClasses = 'Pedestrian', trafficDirections = 'EastBound')
#                                    